In [ ]:
import geopandas as gpd
from datetime import datetime

from metloom.pointdata import MesowestPointData

## READ THIS FOR MESOWEST DATA
https://metloom.readthedocs.io/en/latest/usage.html#mesowest
You'll also need to pip install metloom

In [ ]:

# Import the function to get connect to the db
from snowexsql.db import get_db

# Import our class for the layer data
from snowexsql.data import PointData, SiteData

# Import a useful function to format that data into a dataframe
from snowexsql.conversions import query_to_geopandas

# Import some tools to build dates 
from datetime import date

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

In [ ]:
engine, session = get_db(db_name)
qry = session.query(SiteData.site_name, SiteData.site_id, SiteData.geom)
qry = qry.filter(SiteData.site_name == "Grand Mesa")

gdf_snow_ex = query_to_geopandas(qry, engine) # .drop_duplicates(subset=["site_name"])
session.close()

print(f"The CRS of the Snow Ex metadata is - {gdf_snow_ex.crs}")
gdf_snow_ex_buffer = gdf_snow_ex.buffer(5000)
snow_ex_bounds = gdf_snow_ex.to_crs(4326).total_bounds


In [ ]:
from shapely.geometry import box
def bounds_to_shapefile(bounds, file_name, crs):
    b = [box(*bounds)]
    gdf = gpd.GeoDataFrame(geometry=b).set_crs(crs)
    gdf.to_file(file_name)

In [ ]:
# Specify some user inputs
start_date = datetime(2020, 2, 1)
end_date = datetime(2020, 2, 3)
shape_file_path = "gm_bounds.shp"

In [ ]:
bounds_to_shapefile(snow_ex_bounds, shape_file_path, 4326)
df_shp = gpd.read_file(shape_file_path)
df_shp.plot()
print(df_shp.bounds)

In [ ]:
# variables of interest
var_objs = [MesowestPointData.ALLOWED_VARIABLES.WINDSPEED, MesowestPointData.ALLOWED_VARIABLES.WINDDIRECTION
            ]
# read in the shape file
sf = gpd.read_file(shape_file_path)

# get a collection of points
points = MesowestPointData.points_from_geometry(
    sf, var_objs, within_geometry=False, buffer=0.1
)


In [ ]:
# explore our points
'''
jupyter trust wind_station_finder_and_data_example.ipynb
'''
points_df = points.to_dataframe().set_crs(4326)
print(points_df.crs)
points_map = points_df.explore(tooltip=[i for i in points_df.columns if not i.lower().startswith("geo")])
points_map = sf.explore(m=points_map, style_kwds={"fillOpacity": 0.1})
# plot the map
points_map

In [ ]:
# iterate through points and get hourly data
result_map = {}
for point in points:
    result = point.get_hourly_data(
        start_date, end_date, var_objs
    )
    result_map[point.id] = result
# print(result_map)